What is this about!?

What are tensor-compilers for?
Noticing structure in operations and mapping that onto hardware efficiently?

https://research.fb.com/announcing-tensor-comprehensions/
http://pytorch.org/2018/03/05/tensor-comprehensions.html



[Tensor Comprehension](https://arxiv.org/abs/1802.04730)

* but half the time a single op would be more efficient if it what had come before and what was coming after. so efficient ops is nice, but ...? (is this already handled by traditional compilers?)
* polyhedral intermediate representation
* abstraction without regret
* how much smarter can we make the autotuner? when should it be used?

***

still doesnt get me what i want!? WANT;

* if I have some tensor network representation, I dont want to construct the entire tensor, instead I want to do the ops on the cores/arms.
* keep an online estimate of the total cost of operations, spend compute on finding new faster kernels. 
* ability to take in more context. 
* given some arbitrary tensor operation/representation, find efficient kernels.
* ?

***

https://www.reservoir.com/wp-content/uploads/2017/08/RStreamTF.pdf


In [1]:
import tensor_comprehensions as tc
import torch
lang = """
def tensordot(float(N, C1, C2, H, W) I0, float(N, C2, C3, H, W) I1) -> (O) {
    O(n, c1, c3, h, w) +=! I0(n, c1, c2, h, w) * I1(n, c2, c3, h, w)
}
"""
N, C1, C2, C3, H, W = 32, 512, 8, 2, 28, 28
tensordot = tc.define(lang, name="tensordot")
I0, I1 = torch.randn(N, C1, C2, H, W).cuda(), torch.randn(N, C2, C3, H, W).cuda()
best_options = tensordot.autotune(I0, I1, cache=True)
out = tensordot(I0, I1, options=best_options)

RuntimeError: cuda runtime error (30) : unknown error at /opt/conda/conda-bld/pytorch_1518243271935/work/torch/lib/THC/THCGeneral.c:70

The auto tune function is interesting. 
Keep track of how often a fn is called. Maybe autotune it?